**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

**To do:**
- Add dropout
- Implement another loss function

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
name = 'bpi_2012'
#name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process' 

parser = {
    'train': True,
    'test': True,
    'model_class': 'LSTMAE',
    'model_name': ' ',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.5,
    'input_dir': '../input/{}/'.format(name), 
    'batch_size' : 16,
    'epochs' : 10,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 100,
    'layer2': 100,
    'early_stopping': 30,
    'lr': 0.005,
    'betas': (0.9, 0.99),   
    'lr_decay': 0.99,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}/'.format(name, args.model_class)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

In [12]:
complete_matrix_w_normalized_time_train.shape

(7852, 175, 37)

## Validate and test

In [13]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [14]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [15]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [16]:
missing_true_val.shape, missing_true_test.shape

((55671, 5), (47823, 5))

In [17]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [18]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [19]:
pd.isnull(normalized_missing_df).sum()

CaseID                    0
Activity             130920
CompleteTimestamp    131280
CumTimeInterval      131280
NormalizedTime       131280
dtype: int64

In [20]:
pd.isnull(missing_true_val).sum()

CaseID                   0
Activity             27799
CompleteTimestamp    27961
CumTimeInterval      27961
NormalizedTime       27961
dtype: int64

In [21]:
pd.isnull(missing_true_test).sum()

CaseID                   0
Activity             23817
CompleteTimestamp    23990
CumTimeInterval      23990
NormalizedTime       23990
dtype: int64

## Define model

In [22]:
if args.model_class == 'LSTMAE':
    model = LSTMAE(complete_matrix_w_normalized_time_train.shape[2], args.layer1, 1, args.cuda)
    
if args.cuda:
    model.cuda()

In [23]:
model

LSTMAE (
  (encoder): EncoderRNN (
    (lstm): LSTM(37, 100, batch_first=True)
    (relu): ReLU ()
  )
  (decoder): DecoderRNN (
    (lstm): LSTM(100, 37, batch_first=True)
    (sigmoid): Sigmoid ()
  )
)

## Define loss

In [24]:
# Define loss
def loss_function(recon_x, x, avai_mask): 
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False)
    return BCE

In [25]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [26]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
lambda1 = lambda epoch: args.lr_decay ** epoch
#lambda1 = lambda epoch: args.lr_decay // 1.1
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [27]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [28]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [29]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, nan_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data = model(m_val)
        
    loss = loss_function(recon_data, c_val, nan_matrix_val)
    return loss.data[0]/missing_matrix_w_normalized_time_val.shape[0]

In [30]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,NaN,NaN,NaN
1,7853,NaN,2012-01-05 06:57:15.858,8291911.312,0.0
2,7853,A_DECLINED-COMPLETE,NaN,NaN,NaN
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.0
4,7854,NaN,NaN,NaN,NaN


In [31]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.624972e-03
2,7853,A_DECLINED-COMPLETE,2012-01-05 06:57:55.323,8291950.777,9.974918e-01
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:59:15.103,8292030.557,5.115717e-07


# Train model

In [32]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader,
                                                                complete_matrix_w_normalized_time_trainLoader,
                                                                avai_matrix_trainLoader)):
        
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())
      
        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()
            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [33]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        scheduler.step()
        
        train_loss = train(epoch, model, optimizer)
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        
        #method 2 scheduler
        #scheduler.step(val_score)
        
        end = time.time()
        print('====> Epoch {} | Time: {} ms| Train loss: {:.4f} | Val score: {:.4f}'.format(
                          epoch, (end-init)*1000 ,train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Time: 11062.217712402344 ms| Train loss: 132.4950 | Val score: 136.4790
====> Epoch 2 | Time: 11027.376174926758 ms| Train loss: 129.5201 | Val score: 136.3763
====> Epoch 3 | Time: 10964.017391204834 ms| Train loss: 129.4785 | Val score: 136.3566
====> Epoch 4 | Time: 11007.388353347778 ms| Train loss: 129.4706 | Val score: 136.3548
====> Epoch 5 | Time: 11003.23462486267 ms| Train loss: 129.4673 | Val score: 136.3523
====> Epoch 6 | Time: 11008.719444274902 ms| Train loss: 129.4661 | Val score: 136.3515
====> Epoch 7 | Time: 11011.051654815674 ms| Train loss: 129.4656 | Val score: 136.3513
====> Epoch 8 | Time: 11008.841276168823 ms| Train loss: 129.2968 | Val score: 136.0487
====> Epoch 9 | Time: 11049.046039581299 ms| Train loss: 129.1510 | Val score: 136.0478
====> Epoch 10 | Time: 11026.115894317627 ms| Train loss: 129.1506 | Val score: 136.0470


# Predict and evaluate

In [34]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
        
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    
    print('\n')
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 23990
Mean Absolute Error: 2.0237 day(s)
Root Mean Squared Error: 3.2035 day(s)
Number of missing Activity: 23817
Accuracy: 5.43%


Saving submission...
Done!


In [35]:
submission_df.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.323
2,10470,A_DECLINED-COMPLETE,A_SUBMITTED-COMPLETE,1.086354e+07,1.086351e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:11.148130
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086394e+07,1.086749e+07,2012-02-04 01:24:23.379,2012-02-04 02:23:36.688464
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086749e+07,2012-02-04 01:24:24.052,2012-02-04 02:23:36.697228
7,10471,W_Completeren aanvraag-START,A_SUBMITTED-COMPLETE,1.087644e+07,1.086749e+07,2012-02-04 04:52:43.090,2012-02-04 02:23:36.706788
8,10471,W_Completeren aanvraag-COMPLETE,A_SUBMITTED-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.086749e+07,2012-02-04 05:01:31.267,2012-02-04 02:23:37.442531


In [36]:
submission.head(20)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.323
2,10470,A_DECLINED-COMPLETE,A_SUBMITTED-COMPLETE,1.086354e+07,1.086351e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:11.323
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086394e+07,1.086749e+07,2012-02-04 01:24:23.379,2012-02-04 02:23:36.688464
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086749e+07,2012-02-04 01:24:24.052,2012-02-04 02:23:36.697228
7,10471,W_Completeren aanvraag-START,A_SUBMITTED-COMPLETE,1.087644e+07,1.086749e+07,2012-02-04 04:52:43.090,2012-02-04 02:23:36.706788
8,10471,W_Completeren aanvraag-COMPLETE,A_SUBMITTED-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.087696e+07,2012-02-04 05:01:31.267,2012-02-04 05:01:19.935
